In [4]:
import sys
from pathlib import Path
base_dir = Path.cwd().parent.resolve()
sys.path.append(str(base_dir))
from decorators import Dispatcher

A simple use case of dispatcher is to choose the function based on the type of a selected argument

In [5]:
class UnregisteredKey(KeyError):
    """Error to be raised when an unregistered key is requested"""

def handle_request(value1, value2, value3):
    raise UnregisteredKey('Not a registered key')

# Initialize the dispatcher to dispatch based on the second argument's type (key_idx=1)
handle_request = Dispatcher(handle_request, key_idx=1, key_generator=type)

# Register functions for specific argument types
@handle_request.register(str)
def _(value1, value2, value3):
    return f"String value: {value2}"

@handle_request.register(int)
def _(value1, value2, value3):
    return f"Integer value: {value2}"

@handle_request.register(list)
def _(value1, value2, value3):
    return f"List value: {value2}"

# Dispatch based on the second argument's type
print(handle_request('arg1', 'hello', 'arg3'))  # String value: hello
print(handle_request('arg1', 123, 'arg3'))  # Integer value: 123
print(handle_request('arg1', [1, 2, 3], 'arg3'))  # List value: [1, 2, 3]

# Raises UnregisteredKey for unsupported type
try:
    print(handle_request('arg1', 3.14, 'arg3'))
except UnregisteredKey as ex:
    print(ex)  # Output: Not a registered key

String value: hello
Integer value: 123
List value: [1, 2, 3]
'Not a registered key'


In [6]:
def notify_user(*args, **kwargs):
    raise UnregisteredKey('Not a registered key')

# Key generator function that reverses the values of 'key1' and 'key2'
def reverse_key_order(key1, key2):
    return (key2, key1)

# Initialize the dispatcher to dispatch based on the reversed order of keyword arguments
notify_user = Dispatcher(default_function=notify_user,
                         key_generator=reverse_key_order,
                         key_names=['key1', 'key2'])

# Register functions for specific keys
@notify_user.register(('b', 'a'))
def _(key1, key2):
    return f"Reversed order: {key1}, {key2}"

@notify_user.register(('', None))
def _(key1, key2):
    return f"Empty values: key1={key1}, key2={key2}"

# Dispatch using keyword arguments (key1 and key2), reversing the order of the values
print(notify_user(key1='a', key2='b'))  # Reversed order: b, a
print(notify_user(key1=None, key2=''))  # Empty values: key1=None, key2=

try:
    notify_user('a', key2='b')
except TypeError as ex:
    print(ex)

try:
    notify_user('a', 'b')
except TypeError as ex:
    print(ex)


try:
    print(notify_user(key1='', key2=None))  # Empty values: key1=, key2=None but it will raise an error because the key_generator reverse the order of the keys
except UnregisteredKey as ex:
    print(ex, f'registered_keys: {notify_user.get_registry().keys()}')
# Raises TypeError if a required keyword argument is missing
try:
    print(notify_user())
except ValueError as e:
    print(e)  # Output: Missing required keyword arguments: {'key1', 'key2'}


Reversed order: a, b
Empty values: key1=None, key2=
Missing required keyword arguments: {'key1'}. Arguments used for dispatching must be passed as keyword arguments.
Missing required keyword arguments: {'key1', 'key2'}. Arguments used for dispatching must be passed as keyword arguments.
'Not a registered key' registered_keys: dict_keys([('b', 'a'), ('', None)])
At least one positional or keyword argument is required for dispatching.


In [7]:
def raise_error(*args, **kwargs):
    raise UnregisteredKey('Not a registered key')

class Person:
    def __init__(self, name):
        self.name = name

    talk = Dispatcher(raise_error,
                      key_generator=None,
                      key_idx=1)

    @talk.register('Bob')
    def _(*args):
        return 'Bob'

    @talk.register('Alice')
    def _(*args):
        return 'Alice'

p = Person('No name')
print(p.talk('Alice')) # the first argument is self so it uses the first it was passed
print(p.talk('Bob'))

Alice
Bob


The same example using custom classes

In [8]:
class Dog:
    def __init__(self, name):
        self.name = name

    def talk(self):
        return 'Gab Gab from {}'.format(self.name)

class Cat:
    def __init__(self, name):
        self.name = name

    def talk(self):
        return 'Niaou niaou from {}'.format(self.name)

In [9]:
def talk(value1, value2, value3):
    raise UnregisteredKey('Not a registered key')

talk = Dispatcher(talk ,key_generator=type)

@talk.register(Dog)
def _(obj):
    return obj.talk()

@talk.register(Cat)
def _(obj):
    return obj.talk()

print(talk(Dog('Max')))
print(talk(Dog('Larry')))
print(talk(Cat('Garfield')))

Gab Gab from Max
Gab Gab from Larry
Niaou niaou from Garfield


The default behaviour is to set as key the value of the first argument itself. If this is the case decorator syntax can be used.

In [10]:
@Dispatcher
def registered_key(*args):
    raise UnregisteredKey('Not a registered key')


@registered_key.register('a')
def _value(*args):
    """This is the function for argument a"""
    return registered_key.get_function('a').__doc__

@registered_key.register('b')
def _value(*args):
    """This is the function for argument b"""
    return registered_key.get_function('b').__doc__

@registered_key.register('c')
def _value(*args):
    """This is the function for argument c"""
    return registered_key.get_function('c').__doc__


In [11]:
print(registered_key('a',1))
print(registered_key('b',1))
print(registered_key('c',1))
try:
    registered_key('d',1)
except UnregisteredKey as ex:
    print(ex)

This is the function for argument a
This is the function for argument b
This is the function for argument c
'Not a registered key'


It can be used to decorate methods.

In [12]:
def raise_error(*args, **kwargs):
    raise UnregisteredKey('Not a registered key')

class Person:
    def __init__(self, name):
        self.name = name

    talk = Dispatcher(raise_error,
                      key_generator=None,
                      key_idx=1)

    @talk.register('Bob')
    def _(*args):
        return 'Bob'

    @talk.register('Alice')
    def _(*args):
        return 'Alice'

p = Person('No name')
print(p.talk('Alice')) # the first argument is self so it uses the first it was passed
print(p.talk('Bob'))

Alice
Bob


In [13]:
p.talk # The __get__ method of Dispatcher is called

<bound method raise_error of <__main__.Person object at 0x105564380>>

In [14]:
Person.talk

In [15]:
Person.talk(p, 'Alice')

'Alice'

In [16]:
#Dispatching using the argument name and then pass it as keywrod argument.
def notify_me(*args, **kwargs):
    raise UnregisteredKey('Not a registered key')

def reverse_order(key1, key2):
    return key2, key1

notify_me = Dispatcher(default_function=notify_me,
                       key_generator=reverse_order,
                       key_names=['key1', 'key2'])

@notify_me.register(('b','a'))
def _(key1, key2):
    return key2, key1

@notify_me.register(('', None))
def _(key1, key2):
    return f"{key1=}, {key2=}"
    
# In order to make the mapping of keys with the function I need to pass keyword arguments in the reverse order to match the output of the generator function
print(notify_me(key1='a', key2='b')) # the key generator will reverse the values - > ('b', 'a') just like the values I registered.
print(notify_me(key1=None, key2=''))
try:
    print(notify_me(None, key2=''))
except TypeError as e:
    print(e)

try:
    print(notify_me(None, ""))
except TypeError as e:
    print(e)

try:
    print(notify_me())
except ValueError as e:
    print(e)

('b', 'a')
key1=None, key2=''
Missing required keyword arguments: {'key1'}. Arguments used for dispatching must be passed as keyword arguments.
Missing required keyword arguments: {'key1', 'key2'}. Arguments used for dispatching must be passed as keyword arguments.
At least one positional or keyword argument is required for dispatching.


In [17]:
notify_me.get_registry()

mappingproxy({('b', 'a'): <function __main__._(key1, key2)>,
              ('', None): <function __main__._(key1, key2)>})